**Initialize Ray in Cluster Mode**:

-   Start Ray in cluster mode. This will simulate a cluster environment on your local machine.
-   Use the following code to initialize Ray with two GPUs (assuming your server has them):

In [ ]:
import ray

# Initialize Ray with the dashboard enabled
ray.init(num_gpus=2, include_dashboard=True, dashboard_port=8265)
     
print("Ray initialized with the following resources:")
print(ray.cluster_resources())


**Deploy a Simple Ray Cluster**:

-   Use Ray's API to simulate a cluster environment and run a simple task across multiple nodes.
-   Here's a basic example to demonstrate parallel execution:

In [ ]:
@ray.remote
def compute_square(x):
    return x * x

# Create a list of tasks
tasks = [compute_square.remote(i) for i in range(10)]

# Execute tasks in parallel and gather results
results = ray.get(tasks)
print("Results of parallel computation:", results)


If you're unable to access the Ray dashboard via the default URL (`http://127.0.0.1:8265`), it might be due to network restrictions or firewall settings. Here are a few alternative approaches to access the dashboard:

### **Alternative Ways to Access the Ray Dashboard**

1.  **SSH Tunneling (Port Forwarding)**:
    -   If you're working on a remote server, you can use SSH tunneling to forward the dashboard port to your local machine. This method is secure and doesn't require changing firewall settings.
    -   Run the following command from your local machine (replace `username` and `server_ip` with your server's details):



```

ssh -L 8265:localhost:8265 username@server_ip  

````


-   After setting up the tunnel, open `http://localhost:8265` in your web browser.

---
# Now restart Keneral

**Register and Run a Function in Ray**

Let's use the code you provided earlier to register and run a function in Ray. We'll do this in a Jupyter Notebook cell.

1.  **Register a Function**:
    -   We'll use the `register_function` method from your code to register a function that calculates the mean pixel value of an image.
    -   Here's how you can register the function:

In [1]:
import os
import ray
import uuid
import ast
from pydantic import Field
from typing import Optional

# Initialize Ray
ray.init(num_gpus=2)

# Function registry
function_registry = {}

class ImportMover(ast.NodeTransformer):
    def __init__(self):
        super().__init__()
        self.imports = []

    def visit_Import(self, node):
        self.imports.append(node)
        return None

    def visit_ImportFrom(self, node):
        self.imports.append(node)
        return None

    def visit_FunctionDef(self, node):
        node.body = self.imports + node.body
        return node

def move_imports_inside_functions(code):
    tree = ast.parse(code)
    transformer = ImportMover()
    transformer.visit(tree)
    modified_code = ast.unparse(tree)
    return modified_code

def register_function(
    name: str,
    script: str,
    pip_requirements: Optional[list[str]] = None,
    description: Optional[str] = None
) -> str:
    function_id = str(uuid.uuid4())
    pip_requirements = pip_requirements or []
    script = move_imports_inside_functions(script)
    local_vars = {}
    exec(script, {}, local_vars)
    if 'execute' not in local_vars:
        raise ValueError("The script must define an 'execute' function.")
    execute = ray.remote(runtime_env={"pip": pip_requirements})(local_vars['execute'])
    function_registry[function_id] = {
        "name": name,
        "description": description,
        'pip_requirements': pip_requirements,
        'script': script,
        'execute': execute
    }
    return function_id

# Example function registration
example_pip_requirements = ['numpy', 'requests', 'pillow']
example_script = """
def execute(image_url: str):
    import numpy as np
    from PIL import Image
    import requests
    from io import BytesIO
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content)).convert('L')
    img_array = np.array(img)
    mean_value = np.mean(img_array)
    return float(mean_value)
"""

function_id = register_function(
    name="Mean pixel calculator",
    pip_requirements=example_pip_requirements,
    script=example_script
)
print(f"Registered function with id: {function_id}")


2024-11-04 11:36:13,213	INFO worker.py:1807 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Registered function with id: 85a24be4-a9a5-44f6-8435-af6f425f0335


**Run the Registered Function**:

-   Now that the function is registered, you can run it using the `run_function` method.
-   Here's how you can execute the function:

In [2]:


# Define the async function to run the registered Ray function
async def run_function(
  function_id: str,
  args: Optional[list] = None,
  kwargs: Optional[dict] = None
):
  args = args or []
  kwargs = kwargs or {}
  if function_id not in function_registry:
      raise ValueError(f'Function with id {function_id} not found')
  ray_function = function_registry[function_id]['execute']
  result = ray.get(ray_function.remote(*args, **kwargs))
  return result

# Example of running the function
image_url = "https://ddls.aicell.io/course/ddls-2024/module-1/save-ipynb-file.png"  # Replace with a valid image URL

# Use await directly in the notebook
result = await run_function(function_id, args=[image_url])
print(f"Mean pixel value: {result}")

Mean pixel value: 64.80122377622378


In [1]:

import ray
from ray import tune
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from ray.tune import Callback
# Initialize Ray
ray.init(num_gpus=2)

class CustomCallback(Callback):
    def on_trial_result(self, iteration, trials, trial, result, **info):
        print(f"Trial {trial}: {result}")

# Load dataset
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

def train_model(config):
    # Train a RandomForest model with the given hyperparameters
    model = RandomForestClassifier(n_estimators=config["n_estimators"], max_depth=config["max_depth"])
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    tune.report(accuracy=accuracy)

# Define the search space
search_space = {
    "n_estimators": tune.grid_search([10, 50, 100]),
    "max_depth": tune.choice([5, 10, 15])
}

# Run the hyperparameter tuning
analysis = tune.run(
    train_model,
    config=search_space,
    resources_per_trial={"cpu": 1, "gpu": 1},
    callbacks=[CustomCallback()]
)
print("Best hyperparameters found were: ", analysis.best_config)

2024-11-04 11:53:42,856	INFO worker.py:1807 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2024-11-04 11:53:43,584	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2024-11-04 11:53:45,647	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_0f1cb_00000
Traceback (most recent call last):
  File "/home/scheng/miniconda3/envs/raysam/lib/python3.12/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/scheng/miniconda3/envs/raysam/lib/python3.12/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/scheng/miniconda3/envs/raysam/lib/python3.12/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/scheng/miniconda3/envs/raysam/lib/python3.12/site-packages/ray/_private/worker.py", line 2745, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^

Trial name
train_model_0f1cb_00000
train_model_0f1cb_00001
train_model_0f1cb_00002


2024-11-04 11:53:45,662	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_0f1cb_00001
Traceback (most recent call last):
  File "/home/scheng/miniconda3/envs/raysam/lib/python3.12/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/scheng/miniconda3/envs/raysam/lib/python3.12/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/scheng/miniconda3/envs/raysam/lib/python3.12/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/scheng/miniconda3/envs/raysam/lib/python3.12/site-packages/ray/_private/worker.py", line 2745, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^

TuneError: ('Trials did not complete', [train_model_0f1cb_00000, train_model_0f1cb_00001, train_model_0f1cb_00002])